In [2]:
import random 
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
transition_matrix = pd.read_csv('./data/transition_matrices/transition_matrix_monday.csv', index_col=0)
transition_matrix

,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.089843,0.743920,0.062375,0.051502,0.052361
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.206411,0.087959,0.050821,0.607115,0.047694
spices,0.145765,0.191368,0.176710,0.096906,0.389251


In [4]:
class Customer:
    """
    a single customer that moves through the supermarket
    in a MCMC simulation
    """
    def __init__(self, name, state, budget=100):
        self.name = name
        self.state = state
        self.budget = budget
    def __repr__(self):
        return f'customer_no {self.name} {self.state}'
    def next_state(self):
        '''
        Propagates the customer to the next state.
        Returns nothing.
        '''
        self.state = random.choices(['checkout', 'dairy', 'drinks', 'fruit', 'spices'], list(transition_matrix.loc[self.state]) )
        self.state = self.state[0]
        
        
    def is_active(self):
        
        if self.state == 'checkout':
            return True
        

In [5]:
class Supermarket:
    """manages multiple Customer instances that are currently in the market."""

    def __init__(self,market_name,opening,closing): 
        self.market_name = market_name
        self.opening = opening
        self.closing = closing
        self.customers = []
        self.current_time = 0
        self.index=0
        self.customer_index =0
        self.state = 0
        self.dti = pd.date_range(self.opening, self.closing, freq="T").time
        
        
    def __repr__(self):
        return f'{len(self.dti)}'
    
    def is_open(self):
        if self.index <= len(self.dti)-2:
            return datetime.strptime(self.opening, '%H:%M:%S') <= datetime.strptime(self.get_time(), '%H:%M:%S') <= datetime.strptime(self.closing, '%H:%M:%S')

    def get_time(self):
        """current time in HH:MM format,"""
        self.current_time = self.dti[self.index]
        self.current_time = str(self.current_time)
        return self.current_time
        
    def print_customers(self):
        """print all customers with the current time and id in CSV format.
        """
        return self.customers

    def next_minute(self):
        """"propagates all customers to the next state."""
        
        self.index += 1
        next_time = self.dti[self.index] 
        
        for customer in self.customers:
            customer.next_state()

            
    def add_new_customers(self):
        """randomly creates new customers.
        """
        self.state = random.choices(['dairy', 'drinks', 'fruit', 'spices'])
        self.state = self.state[0]
        self.customer_index +=1
        new_customer = Customer(self.customer_index,self.state)
        self.customers.append(new_customer)
        

    def remove_exitsting_customers(self):
        """removes every customer that is not active any more.
        """
        for customer in self.customers:
            if customer.is_active():
                self.customers.remove(customer)

In [6]:
a = Supermarket('MarketRangers','07:00:00','07:59:00')
a.get_time()
data = {'timestamp': [], 'customer_no': [], 'location': []}
market_rangers = pd.DataFrame(data)
while a.is_open():    
    a.add_new_customers()
    for customer in a.customers:
        print_list = str(a.get_time()+' customer_no '+str(customer.name)+' location '+customer.state)
        print(print_list)
        print_list = print_list.split()
        market_rangers = market_rangers.append({'timestamp': print_list[0], 'customer_no': print_list[2], 'location': print_list[4]},ignore_index=True)
    
    
    a.remove_exitsting_customers()
    a.next_minute()
    

07:00:00 customer_no 1 location spices
07:01:00 customer_no 1 location spices
07:01:00 customer_no 2 location spices
07:02:00 customer_no 1 location checkout
07:02:00 customer_no 2 location drinks
07:02:00 customer_no 3 location dairy
07:03:00 customer_no 2 location drinks
07:03:00 customer_no 3 location checkout
07:03:00 customer_no 4 location dairy
07:04:00 customer_no 2 location drinks
07:04:00 customer_no 4 location dairy
07:04:00 customer_no 5 location fruit
07:05:00 customer_no 2 location checkout
07:05:00 customer_no 4 location dairy
07:05:00 customer_no 5 location fruit
07:05:00 customer_no 6 location fruit
07:06:00 customer_no 4 location dairy
07:06:00 customer_no 5 location checkout
07:06:00 customer_no 6 location fruit
07:06:00 customer_no 7 location dairy
07:07:00 customer_no 4 location dairy
07:07:00 customer_no 6 location drinks
07:07:00 customer_no 7 location dairy
07:07:00 customer_no 8 location dairy
07:08:00 customer_no 4 location dairy
07:08:00 customer_no 6 location

07:38:00 customer_no 31 location dairy
07:38:00 customer_no 33 location fruit
07:38:00 customer_no 37 location checkout
07:38:00 customer_no 38 location checkout
07:38:00 customer_no 39 location dairy
07:39:00 customer_no 9 location checkout
07:39:00 customer_no 19 location dairy
07:39:00 customer_no 27 location drinks
07:39:00 customer_no 31 location dairy
07:39:00 customer_no 33 location dairy
07:39:00 customer_no 38 location checkout
07:39:00 customer_no 39 location dairy
07:39:00 customer_no 40 location dairy
07:40:00 customer_no 19 location fruit
07:40:00 customer_no 27 location drinks
07:40:00 customer_no 31 location dairy
07:40:00 customer_no 33 location checkout
07:40:00 customer_no 39 location checkout
07:40:00 customer_no 40 location fruit
07:40:00 customer_no 41 location spices
07:41:00 customer_no 19 location fruit
07:41:00 customer_no 27 location drinks
07:41:00 customer_no 31 location dairy
07:41:00 customer_no 39 location checkout
07:41:00 customer_no 40 location spices


In [7]:
market_rangers

,timestamp,customer_no,location
0,07:00:00,1,spices
1,07:01:00,1,spices
2,07:01:00,2,spices
3,07:02:00,1,checkout
4,07:02:00,2,drinks
...,...,...,...
365,07:58:00,55,dairy
366,07:58:00,56,fruit
367,07:58:00,57,spices
368,07:58:00,58,fruit
